<a href="https://colab.research.google.com/github/jtneumann/GHwithNLP-SA_LP/blob/master/Week2_growth_hacking_trial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the environment
## Install requirements

In [1]:
!wget https://raw.githubusercontent.com/crow-intelligence/growth-hacking-sentiment/master/requirements.txt
!pip install -r requirements.txt

--2020-06-04 18:47:19--  https://raw.githubusercontent.com/crow-intelligence/growth-hacking-sentiment/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     151  --.-KB/s    in 0s      

2020-06-04 18:47:19 (8.79 MB/s) - ‘requirements.txt’ saved [151/151]

     |████████████████████████████████| 194kB 8.8MB/s 
     |████████████████████████████████| 3.8MB 27.5MB/s 
     |████████████████████████████████| 675kB 54.8MB/s 
     |████████████████████████████████| 204kB 45.3MB/s 
     |████████████████████████████████| 890kB 50.0MB/s 
     |████████████████████████████████| 1.1MB 48.7MB/s 
  Created wheel for Keyness: filename=Keyness-0.25-cp36-none-any.

## Install apex

In [2]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex


Writing setup.sh


In [3]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 7163 (delta 112), reused 86 (delta 55), pack-reused 7012
Receiving objects: 100% (7163/7163), 13.83 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (4828/4828), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-wbscxir_
Created temporary directory: /tmp/pip-req-tracker-34j8dko6
Created requirements tracker '/tmp/pip-req-tracker-34j8dko6'
Created temporary directory: /tmp/pip-install-8y_ayjye
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-9w7zj4tw
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-34j8dko6'
    Running setup.py (

## Connect your Colab notebook to your Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%cd /content/drive/My\ Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [21]:
import os
import sys
import re
import nltk
from nltk.tokenize import word_tokenize, treebank
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import opinion_lexicon
from nltk.text import Text
nltk.download('punkt')
nltk.download('opinion_lexicon')
import pickle



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [7]:

from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn import preprocessing
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import altair as alt


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
!ls

 01_intro.ipynb
 apex
 Building+your+Deep+Neural+Network+-+Step+by+Step+v8.ipynb
'Copy of Colab_growth_hacking_example.ipynb'
'Copy of Week-1 Solution code walk through.ipynb'
'Copy of Week2_growth_hacking_trial2.ipynb'
 data
'DN2_using tabula and confirming readini'
 DN2_week2_trial2
 JT_Colab_gh_Wk-1.ipynb
 JT_Colab_growth_hacking_example2.ipynb
 JT_Colab_growth_hacking_example.ipynb
 setup.sh
 Untitled0.ipynb
 Untitled10.ipynb
 Untitled11.ipynb
 Untitled12.ipynb
 Untitled13.ipynb
 Untitled14.ipynb
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 Untitled4.ipynb
 Untitled5.ipynb
 Untitled6.ipynb
 Untitled7.ipynb
 Untitled8.ipynb
 Untitled9.ipynb
'Week-1 Solution code made to match 1.1.ipynb'
'Week-1 Solution code walk through.ipynb'
 Week2_Colab_growth_hacking_trial1.ipynb
 Week2_growth_hacking_trial3.ipynb
 withOutMaps.ipynb


In [0]:
small_corpus = pd.read_csv('./data/small_corpus.csv')

In [0]:
sc1 = small_corpus.iloc[:,2:]
sc1

NameError: ignored

In [10]:
type(small_corpus['reviews'][0])

str

In [11]:
type(small_corpus['reviews'])

pandas.core.series.Series

In [0]:
lc_small_corpus = [row.reviews.lower() for row in small_corpus.itertuples()]
    

In [13]:
lc_small_corpus[100]

"please hear me out. let me try to explain why i gave this system such a low rating.\n\nhad this console not been remembered as being fantastic and interesting, i would be giving it 3 stars. but, unfortunately, there really isn't anything very interesting about this console. the graphics aren't very good compared to the psx or even some of the better stuff on the saturn, like panzer dragoon saga or virtua fighter 2, the shenmue saturn demo, sega rally, and daytona championship circuit edition, to name a few. lots of games i've played have very low resolution textures, often sprawled out. the games themselves just aren't very interesting or fun to play, even in multiplayer.\n\ni'm also not a huge fan of the controller. while it has an odd design, it is fairly comfortable, and the joystick is very precise.\n\nfinally, there's the value. the games themselves are not at all worth the high prices they demand. super smash brothers simply isn't as fun as powerstone or bushido blade. mario kar

In [0]:
lower_case[100]

"please hear me out. let me try to explain why i gave this system such a low rating.\n\nhad this console not been remembered as being fantastic and interesting, i would be giving it 3 stars. but, unfortunately, there really isn't anything very interesting about this console. the graphics aren't very good compared to the psx or even some of the better stuff on the saturn, like panzer dragoon saga or virtua fighter 2, the shenmue saturn demo, sega rally, and daytona championship circuit edition, to name a few. lots of games i've played have very low resolution textures, often sprawled out. the games themselves just aren't very interesting or fun to play, even in multiplayer.\n\ni'm also not a huge fan of the controller. while it has an odd design, it is fairly comfortable, and the joystick is very precise.\n\nfinally, there's the value. the games themselves are not at all worth the high prices they demand. super smash brothers simply isn't as fun as powerstone or bushido blade. mario kar

In [0]:
positive_words = opinion_lexicon.positive()
negative_words = opinion_lexicon.negative()

In [0]:
sentence_reviews = [sent_tokenize(str(review)) for review in lc_small_corpus ]

In [16]:
sentence_reviews[0]

['yet another garbage cod game.',
 "zombies is unplayable because they're too cheap to give us dedicated servers.",
 'someone leaves?',
 "gg because everyone is getting dc'd.",
 'jetpacks are back and just as lame and boring as ever.',
 'advanced warfare 2.']

In [0]:
def classify_reviews_by_sentence(text):  
    tokenizer = treebank.TreebankWordTokenizer()
    sentiment = 0   
    words = [word.lower() for word in tokenizer.tokenize(text)]
    for i, review in enumerate(text):
        if review in positive_words:
            sentiment += 1
        elif review in negative_words:
            sentiment -= 1
    return sentiment

In [24]:
small_corpus['sentence_sentiment'] = small_corpus['reviews'].apply(classify_reviews_by_sentence)

KeyboardInterrupt: ignored

In [26]:
T = (10,20,30,40,50)
for var in T:
    print (T.index(var),var)

0 10
1 20
2 30
3 40
4 50


In [0]:
small_corpus['sentence_sentiment']

0       0
1       0
2       0
3       0
4       0
       ..
4495    0
4496    0
4497    0
4498    0
4499    0
Name: sentence_sentiment, Length: 4500, dtype: int64

In [0]:
def classify_reviews_by_words(text):  
    sentiment = 0
    reviews = [sentence.lower() for sentence in sent_tokenize(str(text))]
    words = [word for word in word_tokenize(str(reviews))]
    for i, word in enumerate(words):
        if word in positive_words:
            sentiment += 1
        elif word in negative_words:
            sentiment -= 1
    return sentiment

In [0]:
small_corpus['word_sentiment'] = small_corpus['reviews'].apply(classify_reviews_by_words)

KeyboardInterrupt: ignored

In [0]:
small_corpus[small_corpus.word_sentiment==1]

,Unnamed: 0,ratings,reviews,sentence_sentiment,word_sentiment


In [0]:
small_corpus[small_corpus.sentence_sentiment==1]

,Unnamed: 0,ratings,reviews,sentence_sentiment
2089,2089,[3.0],This game is fairly standard hidden-object far...,1
2134,2134,[3.0],"Personally, I do not like this game. It is ve...",1
2151,2151,[3.0],I purchased this as a spare for long trips whe...,1
2461,2461,[3.0],This headset is decent and worked quite well f...,1
2575,2575,[4.0],Game is just like the other Mario and Sonic Ga...,1
2660,2660,[4.0],In this game you play the role of Ryu Hayabusa...,1
2677,2677,[4.0],I had seen this game sitting on the shelves of...,1
2691,2691,[4.0],Good game lots of hard players and computers. ...,1
2746,2746,[4.0],"Excellent game, quick download through Steam c...",1
2853,2853,[4.0],F-Zero GX was surprisingly outsourced to a thi...,1


In [0]:
tokenized_sentences = [sent_tokenize(str(text)) for text in sc]

    

In [0]:
type(tokenized_sentences)

list

In [0]:
tokenized_sentences[0]

['yet another garbage cod game.',
 "zombies is unplayable because they're too cheap to give us dedicated servers.",
 'someone leaves?',
 "gg because everyone is getting dc'd.",
 'jetpacks are back and just as lame and boring as ever.',
 'advanced warfare 2.']

using enumerate to ensure an index, but from the test below, the standard formula above gave the same result.

In [0]:
i_tokenized_sentences = [sent_tokenize(str(review)) for i, review in enumerate(sc)]

In [0]:
i_tokenized_words = [word_tokenize(str(review)) for i, review in enumerate(sc)]

In [0]:
i_tokenized_words[:100]

[['yet',
  'another',
  'garbage',
  'cod',
  'game',
  '.',
  'zombies',
  'is',
  'unplayable',
  'because',
  'they',
  "'re",
  'too',
  'cheap',
  'to',
  'give',
  'us',
  'dedicated',
  'servers',
  '.',
  'someone',
  'leaves',
  '?',
  'gg',
  'because',
  'everyone',
  'is',
  'getting',
  'dc',
  "'d",
  '.',
  'jetpacks',
  'are',
  'back',
  'and',
  'just',
  'as',
  'lame',
  'and',
  'boring',
  'as',
  'ever',
  '.',
  'advanced',
  'warfare',
  '2',
  '.'],
 ['$',
  '80',
  '?',
  '...',
  '.',
  'no',
  'way',
  '.',
  'this',
  'is',
  'not',
  'worth',
  '$',
  '80',
  '.',
  '$',
  '80',
  '?',
  '...',
  '.',
  'no',
  'way',
  '.',
  'this',
  '...'],
 ['one',
  'of',
  'the',
  'worst',
  'games',
  'ever',
  '.',
  'i',
  'bought',
  'and',
  'downloaded',
  'this',
  'from',
  'psn',
  '.',
  'i',
  'do',
  "n't",
  'have',
  'a',
  'uplay',
  'account',
  '.',
  'after',
  'awhile',
  'it',
  "'s",
  'now',
  'stuck',
  'at',
  'the',
  'loading',
  'screen'

In [0]:
type(i_tokenized_sentences)

list

In [0]:
i_tokenized_sentences[0]

['yet another garbage cod game.',
 "zombies is unplayable because they're too cheap to give us dedicated servers.",
 'someone leaves?',
 "gg because everyone is getting dc'd.",
 'jetpacks are back and just as lame and boring as ever.',
 'advanced warfare 2.']

In [0]:
lexicon = np.asarray(i_tokenized_sentences)


In [0]:
type(lexicon)

numpy.ndarray

In [0]:
classification = np.where(lexicon == opinion_lexicon.positive(), 1,
                         np.where(lexicon == opinion_lexicon.negative(), -1, 0))
                                

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [0]:
result = []
for i, word in range(lexicon):
    if word in opinion_lexicon.positive():
        result.append(1)
    elif word in opinion_lexicon.negative():
        result.append(-1)
    else:
        result.append(0)
    if result.sum() > 0:
        lexicon[i] 

In [0]:
classification

array(0)

In [0]:
opinion_lexicon.positive()

['a+', 'abound', 'abounds', 'abundance', 'abundant', ...]

In [0]:
def classify_reviews(reviews):   
    pos_words = 0
    neg_words = 0

    for i, words in enumerate(reviews):
        if words in opinion_lexicon.positive():
            pos_words += 1
        elif words in opinion_lexicon.negative():
            neg_words -= 1
        if pos_words > neg_words:
            words = 1
        elif pos_words < neg_words:
            words = -1
        else:
            words = 0

    return reviews['words']

In [0]:
classification = classify_reviews(i_tokenized_sentences)

TypeError: ignored

In [0]:
classification

0

just another retry from above

In [0]:
def classify_reviews(text):  
    for i, words in enumerate(text):
        count = 0
        if words in opinion_lexicon.positive():
            count += 1
        elif words in opinion_lexicon.negative():
            count -= 1
    return count

In [0]:
def classify_reviews(text):  
    for row in text:
        for words in row:
            count = 0
            if words in opinion_lexicon.positive():
                count += 1
            elif words in opinion_lexicon.negative():
                count -= 1
    return count

In [0]:
l = classify_reviews(i_tokenized_sentences)

KeyboardInterrupt: ignored

In [0]:
l

0

In [0]:
count = itertools.count()
test = [c += 1 for i, words in enumerate(i_tokenized_sentences) if words in opinion_lexicon.positive() ]

SyntaxError: ignored

In [0]:
i_tokenized_sentences[0]

['yet another garbage cod game.',
 "zombies is unplayable because they're too cheap to give us dedicated servers.",
 'someone leaves?',
 "gg because everyone is getting dc'd.",
 'jetpacks are back and just as lame and boring as ever.',
 'advanced warfare 2.']

In [0]:
tokenized_sentences = [sent_tokenize(str(sc['reviews'][i])) for i in sc]

KeyError: ignored

2.1 says to download in this format, but comes up bool when doing so; problem resolved by importing opinion_lexicon instead, which returns a LexiconCorpusReader

In [0]:
type(opinion_lexicon)

nltk.corpus.reader.opinion_lexicon.OpinionLexiconCorpusReader

In [0]:
opinion_lexicon.negative()

['2-faced', '2-faces', 'abnormal', 'abolish', ...]

In [0]:
opinion_lexicon.positive()

['a+', 'abound', 'abounds', 'abundance', 'abundant', ...]

In [0]:
# tokenize to words and convert to lower case
words = word_tokenize(str(sentences_wo_punctuation))
tokenized_words = [word.lower() for word in words]

In [0]:
tokenized_words[:100]

['[',
 "'",
 ',',
 'ratings',
 ',',
 'reviews\\n0',
 ',',
 '[',
 '1.0',
 ']',
 ',',
 'yet',
 'anothegarbage',
 'cod',
 'game',
 '.',
 "'",
 ',',
 '``',
 'zombies',
 'is',
 'unplayable',
 'because',
 'they',
 "'re",
 'too',
 'cheap',
 'to',
 'give',
 'us',
 'dedicated',
 'servers',
 '.',
 '``',
 ',',
 "'someone",
 'leaves',
 '?',
 "'",
 ',',
 '``',
 'gg',
 'because',
 'everyone',
 'is',
 'getting',
 'dc',
 "'d",
 '.',
 '``',
 ',',
 "'jetpacks",
 'are',
 'back',
 'and',
 'just',
 'as',
 'lame',
 'and',
 'boring',
 'as',
 'eve',
 "'",
 ',',
 "'advanced",
 'warfare',
 '2',
 '.',
 "'",
 ',',
 "'1",
 ',',
 '[',
 '1.0',
 ']',
 ',',
 '$',
 '80',
 '?',
 "'",
 ',',
 "'",
 '...',
 '.',
 'no',
 'way',
 '.',
 "'",
 ',',
 "'this",
 'is',
 'not',
 'worth',
 '$',
 '80',
 '.',
 "'",
 ',',
 "'",
 '$']

In [0]:
tokenized_words.words(fi)

list

In [0]:
def count_lexicon(sentence, plot=False):
    """
    :param sentence: a sentence whose polarity has to be classified.
    :param plot: if True, plot a visual representation of the sentence polarity.
    """
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in sent_tokenize(sentence)]

    x = list(range(len(tokenized_sent))) # x axis for the plot
    y = []

    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
            y.append(1) # positive
        elif word in opinion_lexicon.negative():
            neg_words += 1
            y.append(-1) # negative
        else:
            y.append(0) # neutral

    if pos_words > neg_words:
        print('Positive', y.count(pos_words))
    elif pos_words < neg_words:
        print('Negative', y.count(neg_words))
    elif pos_words == neg_words:
        print('Neutral', y.count(pos_words))


In [0]:
type(sentences_wo_punctuation)

list

In [0]:
pos_w = opinion_lexicon.positve()

AttributeError: ignored

In [0]:

count_lexicon(str(sentences_wo_punctuation))

Neutral 34891


In [0]:
count_lexicon(str(words))

Neutral 36514


In [0]:
sentences_wo_punctuation_freqDist = nltk.FreqDist(sentences_wo_punctuation)
sentences_wo_punctuation_freqDist.most_common(25)

[('2.', 39),
 ('3.', 33),
 ('.', 32),
 ('4.', 26),
 ('5.', 20),
 ('6.', 17),
 ('1.', 15),
 ('Why?', 9),
 ('7.', 9),
 ('Check.', 9),
 ('Yes.', 8),
 ('I was wrong.', 6),
 ('Terrible.', 5),
 ('8.', 5),
 ('I love it.', 5),
 ("That's it.", 5),
 ('Seriously.', 5),
 ('Horrible.', 5),
 ('Great game!"', 5),
 ('Nope.', 4),
 ('Save youmoney.', 4),
 ('Wow!', 4),
 ("You won't be disappointed.", 4),
 ('Rev.', 4),
 ('Thanks!', 4)]

In [0]:
word_features = list(sentences_wo_punctuation_freqDist.keys())

In [0]:
len(word_features)

36243

NLTK Tutorial.

In [0]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [0]:
print(find_features())

featuresets = [ (find_features(review), category) for (review, category) in documents]

In [0]:
training_set = featuresets[:1900]
testing_set = featuresets[1900:]

In [0]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Naive Bayes Acc:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

In [0]:
save_classifier = open('naivebayes.picke', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [0]:
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [0]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

In [0]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000] # [_document-classify-all-words]

def document_features(document): # [_document-classify-extractor]
    document_words = set(document) # [_document-classify-set]
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [0]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [0]:
print(nltk.classify.accuracy(classifier, test_set))

0.78


In [0]:
classifier.show_most_informative_features(5)

Most Informative Features
    contains(recognizes) = True              pos : neg    =      8.1 : 1.0
 contains(unimaginative) = True              neg : pos    =      7.8 : 1.0
    contains(schumacher) = True              neg : pos    =      7.8 : 1.0
        contains(turkey) = True              neg : pos    =      6.5 : 1.0
     contains(atrocious) = True              neg : pos    =      6.4 : 1.0


In [0]:
movie_reviews.categories()

['neg', 'pos']

In [0]:
nltk.corpus.reader.wordlist.NonbreakingPrefixesCorpusReader(str(words(id)))

TypeError: ignored